In [1]:
from IPython.display import Audio
from datasets import load_dataset, load_from_disk
import os
import gc
import shutil
import torch
from scipy.io import wavfile
import numpy as np
from pydub import AudioSegment
from scipy import signal

In [ ]:
source_dir = "//kaggle/input/mathbridge-filtered"

# Destination directory
destination_dir = "/kaggle/working/mathbridge-filtered"

shutil.copytree(source_dir, destination_dir)

In [ ]:
ds_train = load_from_disk("/kaggle/working/mathbridge-filtered/mathbridge_filtered")
ds_train

In [ ]:
# Define a function to map the text to a file path
def map_to_speech_file(example, idx):
    if idx%100==0:
        print(idx)
    filepath = f"/kaggle/input/mathbridge-audio-files/speech/tts_{idx}.mp3"
    try:
        audio_array = wavfile.read(filepath)[1]
        num_samples = int(len(audio_array) * 16000 / 22050)
        new_audio_array = signal.resample(audio_array, num_samples)
        example["file_name"] = f"speech/tts_{idx}.mp3"
        example["speech_array"] = new_audio_array
        del audio_array, num_samples, new_audio_array
        gc.collect()
    except:
        example["file_name"] = None
        example["speech_array"] = None
    return example

# Apply the function to the dataset
ds_speech = ds_train.select(range(40000)).map(map_to_speech_file, with_indices=True)

In [ ]:
ds_speech[1]

In [ ]:
ds_speech.save_to_disk("/kaggle/working/mathbridge_speech")

In [ ]:
shutil.rmtree("/kaggle/working/mathbridge-filtered")
# os.remove("/kaggle/working/state.db")